# ⌖ Technical Indicators
In this notebook we are calculating 12 different technical indicators. There are classified in 4 types of indicators: tendency indicators, momentum indicators, volatility indicators and volume indicators. There exist a lot more indicators, but firstly we are calculating those basic ones.  
 
We will first calculate the indicators for the 1h timeframe dataset.

## Importing the dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('../data/btcusdt_1h_2018_preprocessed.csv')
df.head()

,Date,Open,High,Low,Close,Volume
0,2018-01-01 00:00:00,13715.65,13715.65,13400.01,13529.01,443.356199
1,2018-01-01 01:00:00,13528.99,13595.89,13155.38,13203.06,383.697006
2,2018-01-01 02:00:00,13203.00,13418.43,13200.00,13330.18,429.064572
3,2018-01-01 03:00:00,13330.26,13611.27,13290.00,13410.03,420.087030
4,2018-01-01 04:00:00,13434.98,13623.29,13322.15,13601.01,340.807329


## 📉 Tendency indicators
They indicate the general direccion of the price.

### Moving Average (EMA and SMA)